<a href="https://colab.research.google.com/github/teamgaon/KLUE/blob/main/220222_sm_TPU_kfold_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# A full training

Install the Transformers and Datasets libraries to run this notebook.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install datasets transformers[sentencepiece]
!pip install accelerate
# To run the training on TPU, you will need to uncomment the followin line:
!pip install cloud-tpu-client==0.10 torch==1.9.0 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl
!pip install transformers

     |████████████████████████████████| 311 kB 5.3 MB/s 
     |████████████████████████████████| 3.5 MB 86.1 MB/s 
     |████████████████████████████████| 1.1 MB 67.3 MB/s 
     |████████████████████████████████| 67 kB 5.0 MB/s 
     |████████████████████████████████| 243 kB 98.3 MB/s 
     |████████████████████████████████| 133 kB 66.8 MB/s 
     |████████████████████████████████| 271 kB 93.8 MB/s 
     |████████████████████████████████| 144 kB 82.5 MB/s 
     |████████████████████████████████| 94 kB 3.8 MB/s 
     |████████████████████████████████| 895 kB 82.3 MB/s 
     |████████████████████████████████| 6.8 MB 81.2 MB/s 
     |████████████████████████████████| 596 kB 87.1 MB/s 
     |████████████████████████████████| 1.2 MB 87.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 58 kB 3.1 MB/s 
     |████████████████████████████████| 149.9 

In [ ]:
from accelerate import Accelerator

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import random
from tqdm import tqdm
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from transformers import TrainingArguments, Trainer
from transformers import AutoModel,AutoModelForSequenceClassification, AutoConfig, AutoTokenizer
import gc
from transformers import AdamW
from transformers import get_scheduler, get_cosine_with_hard_restarts_schedule_with_warmup
from tqdm.auto import tqdm
from datasets import load_metric
import warnings
warnings.filterwarnings('ignore')
from accelerate import notebook_launcher

In [ ]:
kakao_snli = pd.read_csv('/content/drive/MyDrive/KLUE/snli_1.0_train.ko.tsv', sep='\t', encoding='utf-8')
kakao_snli = kakao_snli[:100000]

PATH =  '/content/drive/MyDrive/KLUE'
test = pd.read_csv(os.path.join(PATH, 'test_data.csv'), encoding='utf-8')

kakao_dev = pd.read_csv('/content/drive/MyDrive/KLUE/xnli.dev.ko.tsv', sep='\t', encoding='utf-8')
kakao_test = pd.read_csv('/content/drive/MyDrive/KLUE/xnli.test.ko.tsv', sep='\t', encoding='utf-8')
klue_dev = pd.read_json('/content/drive/MyDrive/KLUE/klue-nli-v1.1_dev.json')
klue_train = pd.read_json('/content/drive/MyDrive/KLUE/klue-nli-v1.1_train.json')
temp = pd.read_csv(os.path.join(PATH, 'train_data.csv'), encoding='utf-8')

klue_dev = klue_dev[['premise', 'hypothesis', 'gold_label']]
klue_train = klue_train[['premise', 'hypothesis', 'gold_label']]
klue_dev.rename(columns = {'gold_label':'label'}, inplace=True)
klue_train.rename(columns = {'gold_label':'label'}, inplace=True)

train = pd.concat([kakao_dev,kakao_test, kakao_snli])
train.rename(columns = {'sentence1':'premise','sentence2':'hypothesis','gold_label':'label'},inplace=True)
train = pd.concat([temp, train, klue_dev, klue_train], axis=0)
train = train.reset_index(drop=True)
train['index'] = train.index
train = train.dropna()
train

In [ ]:
from transformers import AutoTokenizer

checkpoint = 'klue/roberta-large'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [ ]:
# train_dataset, eval_dataset = train_test_split(train, test_size=0.2, shuffle=False)

# tokenized_train = tokenizer(
#     list(train_dataset['premise']),
#     list(train_dataset['hypothesis']),
#     return_tensors="pt",
#     max_length=128, # Max_Length = 190
#     padding=True,
#     truncation=True,
#     add_special_tokens=True
# )

# tokenized_eval = tokenizer(
#     list(eval_dataset['premise']),
#     list(eval_dataset['hypothesis']),
#     return_tensors="pt",
#     max_length=128,
#     padding=True,
#     truncation=True,
#     add_special_tokens=True
# )

In [ ]:
class BERTDataset(torch.utils.data.Dataset):
    def __init__(self, pair_dataset, label):
        self.pair_dataset = pair_dataset
        self.label = label

    def __getitem__(self, idx):
        item = {key: val[idx].clone().detach() for key, val in self.pair_dataset.items()}
        item['labels'] = torch.tensor(self.label[idx])
        
        return item

    def __len__(self):
        return len(self.label)

In [ ]:
def label_to_num(label):
    label_dict = {"entailment": 0, "contradiction": 1, "neutral": 2, "answer": 3}
    num_label = []

    for v in label:
        num_label.append(label_dict[v])
    
    return num_label

# train_label = label_to_num(train['label'].values)
# eval_label = label_to_num(eval_dataset['label'].values)

In [ ]:
# train_dataset = BERTDataset(tokenized_train, train_label)
# eval_dataset = BERTDataset(tokenized_eval, eval_label)

In [ ]:
# train_dataset[0]

In [ ]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [ ]:
# def training_function():
#   accelerator = Accelerator()

#   train_dataloader = torch.utils.data.DataLoader(
#                     train_dataset, 
#                     batch_size=16, sampler=train_subsampler, num_workers=2)
#   eval_dataloader = torch.utils.data.DataLoader(
#                     train_dataset,
#                     batch_size=16, sampler=test_subsampler, num_workers=2)

#   config = AutoConfig.from_pretrained(checkpoint)
#   config.num_labels = 3
#   model = AutoModelForSequenceClassification.from_pretrained(checkpoint, config=config)

#   optimizer = AdamW(model.parameters(), lr=	1e-5)

#   model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(model, optimizer, train_dataloader, eval_dataloader)

#   num_epochs = 5
#   num_training_steps = num_epochs * len(train_dataloader)
#   progress_bar = tqdm(range(num_training_steps))
#   lr_scheduler = get_cosine_with_hard_restarts_schedule_with_warmup(
#       optimizer=optimizer,
#       num_warmup_steps=1,
#       num_training_steps=num_training_steps,
#   )

#   for epoch in range(num_epochs):
#       train_acc = 0.0
#       test_acc = 0.0

#       model.train()
#       for batch_id, batch in enumerate(train_dataloader):
#           outputs = model(batch['input_ids'], batch['attention_mask'], batch['token_type_ids'])
#           loss = F.cross_entropy(outputs[0], batch['labels'])
#           accelerator.backward(loss)

#           optimizer.step()
#           lr_scheduler.step()
#           optimizer.zero_grad()
#           progress_bar.update(1)
#           train_acc += calc_accuracy(outputs.logits, batch['labels'])
#       print("epoch {} train acc {}".format(epoch+1, train_acc / (batch_id+1)))

#       model.eval()
#       for batch_id, batch in enumerate(eval_dataloader):
#         with torch.no_grad():
#             outputs = model(batch['input_ids'], batch['attention_mask'], batch['token_type_ids'])

#         test_acc += calc_accuracy(outputs.logits, batch['labels'])
#       print("epoch {} test acc {}".format(epoch+1, test_acc / (batch_id+1)))
#       gc.collect()
#   accelerator.wait_for_everyone()
#   unwrapped_model = accelerator.unwrap_model(model)
#   accelerator.save(unwrapped_model.state_dict(), '/content/drive/MyDrive/KLUE/model.pt')

In [ ]:
kfold = StratifiedKFold(n_splits=5, shuffle=True)

# Start print
print('--------------------------------')

# K-fold Cross Validation model evaluation
def training_function():
  kfold = StratifiedKFold(n_splits=5, shuffle=True)

  print('--------------------------------')

  for fold, (train_ids, test_ids) in enumerate(kfold.split(train, train['label'])):
    print('')
    print(f'FOLD {fold}')
    print('--------------------------------')

    train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
    test_subsampler = torch.utils.data.SubsetRandomSampler(test_ids)

    train_label = label_to_num(train['label'].values)

    tokenized_train = tokenizer(
      list(train['premise']),
      list(train['hypothesis']),
      return_tensors="pt",
      max_length=128, # Max_Length = 190
      padding=True,
      truncation=True,
      add_special_tokens=True
      )
    
    train_dataset = BERTDataset(tokenized_train, train_label)

    # def training_function():
    accelerator = Accelerator()

    train_dataloader = torch.utils.data.DataLoader(
                      train_dataset, 
                      batch_size=16, sampler=train_subsampler, num_workers=2)
    eval_dataloader = torch.utils.data.DataLoader(
                      train_dataset,
                      batch_size=16, sampler=test_subsampler, num_workers=2)

    config = AutoConfig.from_pretrained(checkpoint)
    config.num_labels = 3
    model = AutoModelForSequenceClassification.from_pretrained(checkpoint, config=config)

    optimizer = AdamW(model.parameters(), lr=	1e-5)

    model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(model, optimizer, train_dataloader, eval_dataloader)

    num_epochs = 5
    num_training_steps = num_epochs * len(train_dataloader)
    progress_bar = tqdm(range(num_training_steps))
    lr_scheduler = get_cosine_with_hard_restarts_schedule_with_warmup(
        optimizer=optimizer,
        num_warmup_steps=1,
        num_training_steps=num_training_steps,
    )

    for epoch in range(num_epochs):
        train_acc = 0.0
        test_acc = 0.0

        model.train()
        for batch_id, batch in enumerate(train_dataloader):
            outputs = model(batch['input_ids'], batch['attention_mask'], batch['token_type_ids'])
            loss = F.cross_entropy(outputs[0], batch['labels'])
            accelerator.backward(loss)

            optimizer.step()
            lr_scheduler.step()
            optimizer.zero_grad()
            progress_bar.update(1)
            train_acc += calc_accuracy(outputs.logits, batch['labels'])
        print("epoch {} train acc {}".format(epoch+1, train_acc / (batch_id+1)))

        model.eval()
        for batch_id, batch in enumerate(eval_dataloader):
          with torch.no_grad():
              outputs = model(batch['input_ids'], batch['attention_mask'], batch['token_type_ids'])

          test_acc += calc_accuracy(outputs.logits, batch['labels'])
        print("epoch {} test acc {}".format(epoch+1, test_acc / (batch_id+1)))
        gc.collect()
    accelerator.wait_for_everyone()
    unwrapped_model = accelerator.unwrap_model(model)
    accelerator.save(unwrapped_model.state_dict(), '/content/drive/MyDrive/KLUE/model' + str(fold) + '.pt')

In [ ]:
notebook_launcher(training_function)

In [ ]:
config = AutoConfig.from_pretrained(checkpoint)
config.num_labels = 3
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, config=config)

In [ ]:
accelerator = Accelerator()
model = accelerator.unwrap_model(model)
model.load_state_dict(torch.load('/content/drive/MyDrive/model/model.pt'))

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [ ]:
test_label = label_to_num(test['label'].values)

tokenized_test = tokenizer(
    list(test['premise']),
    list(test['hypothesis']),
    return_tensors="pt",
    max_length=128,
    padding=True,
    truncation=True,
    add_special_tokens=True
)

test_dataset = BERTDataset(tokenized_test, test_label)

In [ ]:
dataloader = DataLoader(test_dataset, batch_size=16, shuffle=False)

output_pred = []
output_prob = []

model, dataloader= accelerator.prepare(model, dataloader)

model.eval()

for i, data in enumerate(tqdm(dataloader)):
    with torch.no_grad():
        outputs = model(
            input_ids=data['input_ids'],
            attention_mask=data['attention_mask'],
            token_type_ids=data['token_type_ids']
        )
    logits = outputs[0]
    prob = F.softmax(logits, dim=-1).detach().cpu().numpy()
    logits = logits.detach().cpu().numpy()
    result = np.argmax(logits, axis=-1)

    output_pred.append(result)
    output_prob.append(prob)
  
pred_answer, output_prob = np.concatenate(output_pred).tolist(), np.concatenate(output_prob, axis=0).tolist()
print(pred_answer)

In [ ]:
def num_to_label(label):
    label_dict = {0: "entailment", 1: "contradiction", 2: "neutral"}
    str_label = []

    for i, v in enumerate(label):
        str_label.append([i,label_dict[v]])
    
    return str_label

answer = num_to_label(pred_answer)
print(answer)

In [ ]:
df = pd.DataFrame(answer, columns=['index', 'label'])

df.to_csv('/content/drive/MyDrive/KLUE/submission.csv', index=False)

print(df)